In [ ]:
import numpy as np
import pandas as pd
import glob
import time, datetime
import random
import sklearn.metrics
from sklearn.neural_network import MLPRegressor
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.externals import joblib

# Загружаем данные из Excel-файлов

In [ ]:
# задаём путь к папке, где лежат данные
folder_name = 'D:\\Users\\AATimchenko\\Documents\\Doc-Projects\\RDF\\Моделирование расчета TS\\NeuralNetworkForecast\\'

# определяем список столбцов c с параметрами, которые оставляем для использования в прогнозе
cols_to_retain = ['Подкатегория','Товарная подгруппа','IntakeWeek','ExitWeek','NSize','Пол','ТМ','Возраст 1','Товар драйвер']

# эти колонки для прогноза не нужны, но мы их оставляем, т.к. будем их использовать для анализа точности прогнозов
id_cols = ['Цветомодель', 'Наименование']

# задаем количество прошлых недель, по которым будем строить прогноз
#weeks = ['w6','w5','w4','w3','w2','w1']
#query_not_zero_weeks = 'w6 > 0 or w5 > 0 or w4 > 0 or w3 > 0 or w2 > 0 or w1 > 0'
w = 6
weeks = ['w'+str(i) for i in range (w,0,-1)]
query_not_zero_weeks = (' > 0 or ').join(weeks) + ' > 0'


In [ ]:
# загружаем данные по IntakeDate и ExitDate из Excel-файла (данные брал из ПРК)
file_name_data = folder_name + 'IntakeDates.xlsx'
df_dates = pd.read_excel(io = file_name_data)
df_dates_cols = ['ID','NSize','IntakeDate','ExitDate']
df_dates = df_dates [df_dates_cols]

print(df_dates.shape)
df_dates.head()

In [ ]:
# функция для преобразования данных о продажах из Excel-файлов, выгруженных из ТМА и ПРК, в нужный формат
def preprocess_data (data, df_dates):

    cols_to_read = ['Цветомодель',
                    'Наименование',
                    'Группа товаров',
                    'Подкатегория',
                    'Товарная группа',
                    'Товарная подгруппа',
                    'Пол',
                    'Возраст 1',
                    'ТМ',
                    'Коллекция',
                    'Тип товара',
                    'Товар драйвер',
                    'Менеджер']
    
    for x in data.columns:
        if x not in cols_to_read and type(x) != datetime.datetime: 
            data.drop (x, axis = 1, inplace = True)

    data.dropna(axis=0, how='any', thresh=None, subset=['Группа товаров'], inplace=True)
    data = data.assign(ID = data['Цветомодель'] + '=' + data['Коллекция'])
    
    data = data.set_index('ID').join(df_dates.set_index('ID'))
    
    data.dropna(axis=0, how='any', thresh=None, subset=['IntakeDate','ExitDate'], inplace=True)
    data['Товар драйвер'].fillna(value = 'Нет', inplace = True)
    
    # считаем и добавляем столбцы IntakeWeek и ExitWeek - плановая неделя начала продаж
    data = data.assign(IntakeWeek = lambda x: [x.isocalendar()[1] for x in data['IntakeDate']])
    data = data.assign(ExitWeek = lambda x: [x.isocalendar()[1] for x in data['ExitDate']])

    # получаем список столбцов, в которые данные о продажах (у них в шапках дата)
    week_cols = [x for x in data.columns if type(x) == datetime.datetime]
   
    new_data = pd.DataFrame()
    
    for i in range(0,len(week_cols)-len(weeks)):
        
        # берем w+1 столбцов с продажами
        add = data[week_cols].iloc[:,i:i+len(weeks)+1]

        # переименовываем столбцы
        #add.columns= ['w4','w3','w2','w1','w0']
        add.columns= weeks + ['w0']

        # добавляем столбец с номером последней недели
        add.loc [:,'WeekNumber']= np.array([data[week_cols].columns[i+len(weeks)].isocalendar()[1]]*len(add))

        # добавляем атрибуты товара и добавляем в целевую таблицу
        new_data = new_data.append(add.merge(data[id_cols+cols_to_retain], left_index = True, right_index = True), ignore_index=True)

    #удаляем наборы данных, где продажи во всех прошлых неделях нулевые
    new_data = new_data.query(query_not_zero_weeks)

    return new_data

In [ ]:
# загружаем понедельные данные о продажах из excel
file_name_data = folder_name + 'AppDataForNN*'
data = pd.DataFrame()

for f in glob.glob(file_name_data):
    print (f)
    df = pd.read_excel(io = f, sheetname = 'Продажи-ШТ')
    data = data.append(preprocess_data (df,df_dates))
    
# Сохраняем считанное в файл, нужно для отладки чтобы потом много раз не читать excel
joblib.dump(data, 'data.pkl')

print(data.shape)
data.head()


In [ ]:
# выгрузка подготовленных данных в файл для теста
#file_test = folder_name + 'TestData.xlsx'
#data.to_excel(excel_writer=file_test, sheet_name='Sheet1',startrow=0, startcol=0)
# ===

# Преобразовываем загруженные данные 

In [ ]:
# загружаем исходные данные из дампа
#data = joblib.load('data.pkl')

In [ ]:
# Оставляем только нужные подкатегории
#data = data.query('Подкатегория == "Спортивный стиль" or Подкатегория == "Basic Sport" or Подкатегория == "Outdoor Core" or Подкатегория == "Outdoor Style"')
data = data.query('Подкатегория == "Outdoor Core" or Подкатегория == "Outdoor Style"')

#сохраняем отфильтрованные данные, далее к ним допишем прогноз
filtered_data = data
joblib.dump(filtered_data, 'filtered_data.pkl')

# удаляем столбцы с идентификаторами товара (Цветомодель и Наименование)
train = filtered_data.drop (id_cols, axis = 1)
#train.info()


In [ ]:
# функция преобразует текстовые параметры в числовые столбцы, с которыми работает прогнозная модель
def transform_data (data = None, vectorizer = None, max_data = None):
    
    # numeric x - создаем матрицу с численными параметрами
    numeric_cols = weeks + ['NSize']
    x_num_data = data[numeric_cols].as_matrix()

    # scale to <0,1> - нормируем численные параметры от 0 до 1
    if max_data is None: max_data = np.amax( x_num_data, 0 )
    x_num_data = x_num_data / max_data

    # y
    y_data = data.w0

    # categorical - берем только категорийные параметры
    cat_data = data.drop( numeric_cols + ['w0'] , axis = 1 )
    cat_data.fillna( 'NA', inplace = True )

    x_cat_data = cat_data.to_dict( orient = 'records' )

    # vectorize - векторизуем категорийные параметры
    if vectorizer is None :
        vectorizer = DV( sparse = False )
        vec_x_cat_data = vectorizer.fit_transform( x_cat_data )
    else:
        vec_x_cat_data = vectorizer.transform( x_cat_data )

    # complete x - соединяем вместе численные столбцы и векторизированные категорийные столбцы
    x_data = np.hstack(( x_num_data, vec_x_cat_data ))
    
    return x_data, y_data, max_data, vectorizer

In [ ]:
# X - матрица для обучения, y - вектор правильных ответов
X, y, max_data, vectorizer = transform_data (data = train, vectorizer = None, max_data = None)

# сохраняем промежуточные результаты в файлы
joblib.dump(X, 'x.pkl')
joblib.dump(y, 'y.pkl')
joblib.dump(max_data, 'max_data.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

# размерность матрицы входящих данных для сети
X.shape

# Обучаем сеть, параметры выбираем вручную

In [ ]:
# фиксируем генератор случайных чисел, нужно для повторяемости результатов
seed = 24
np.random.seed(seed)

# задаем параметры сети вручную
# число нейронов в скрытых слоях делаю равным удвоенному количеству входных нейронов (потом нужно подбирать)
# n = X.shape[1]*2

# число слоев и нейронов
hls = ( X.shape[1]*2, X.shape[1]*2, ) # показала хороший результат R2 = 0.82
# hls = (100,100,)                   # показала хороший результат R2 = 0.81


# задаем модель
MLP = MLPRegressor (
                    activation = 'logistic',       # функция активации 
                    hidden_layer_sizes = hls, 
                    alpha = 0.0001,                # регуляризация default = 0.0001
                    warm_start = False, 
                    verbose = True, 
                    tol = 0.00001)                 # Tolerance for the optimization default = 0.0001

# делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2, random_state=seed)

# обучаем сеть на тестовых данных, замеряем время
MLP_start = time.perf_counter()
MLP.fit (X_train, y_train)
MLP_end = time.perf_counter()

# сохраняем обученную сеть в файл
joblib.dump(MLP, 'mlp.pkl')

# смотрим результаты
print('=============================')
print(MLP.get_params())
print('=============================')
print('Time to fit: ',(MLP_end - MLP_start))
print('In-Sample R2_score: ', MLP.score(X_train, y_train))
print('Out-of-Sample R2_score: ', MLP.score(X_test, y_test))


# расчёт прогноза по тестовой выборке Out-of-Sample
MLP_start = time.perf_counter()
forecast = MLP.predict(X_test)
MLP_end = time.perf_counter()

print('=============================')
print("Time to predict Out-of-Sample: {0}".format(MLP_end - MLP_start))
print("Out-of-Sample R2_score: {0}".format(sklearn.metrics.r2_score(y_true=y_test, y_pred=forecast)))
print("mean_absolute_error: {0}".format(sklearn.metrics.mean_absolute_error(y_true=y_test, y_pred=forecast)))
print("mean_squared_error: {0}".format(sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=forecast)))
print("median_absolute_error: {0}".format(sklearn.metrics.median_absolute_error(y_true=y_test, y_pred=forecast)))

In [ ]:
# Если результаты ОК, то можно переобучить сеть по всем имеющимся данным
MLP_start = time.perf_counter()
MLP.fit(X, y)
MLP_end = time.perf_counter()

# сохраняем обученную сеть в файл
joblib.dump(MLP, 'mlp.pkl')

# смотрим результаты
print('Time: ', (MLP_end - MLP_start))
print('All data R2_score: ',MLP.score(X, y))

# Расчет прогноза и запись результатов в файл

In [ ]:
# если исходные данные для прогноза нужно загрузить из Excel
#file_test = folder_name + 'TestData.xlsx'
#test = pd.read_excel(io = file_name_data)

# исходные данные
test = filtered_data

# Оставляем только нужные столбцы
test = test[weeks + ['w0','WeekNumber'] + cols_to_retain]

# преобразовываем данные к нужному виду
X_test, y_test, vectorizer, max_data = transform_data (data = test, vectorizer = vectorizer, max_data = max_data)

#расчёт прогноза
MLP_start = time.perf_counter()
forecast = MLP.predict(X_test)
MLP_end = time.perf_counter()

print("Time: {0}".format(MLP_end - MLP_start))
print("R2_score: {0}".format(sklearn.metrics.r2_score(y_true=y_test, y_pred=forecast)))
print("mean_absolute_error: {0}".format(sklearn.metrics.mean_absolute_error(y_true=y_test, y_pred=forecast)))
print("mean_squared_error: {0}".format(sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=forecast)))
print("median_absolute_error: {0}".format(sklearn.metrics.median_absolute_error(y_true=y_test, y_pred=forecast)))

In [ ]:
# исходные данные с идентификационными столбцами
result = filtered_data

# добавляем к исходным данным расчитанный прогноз и считаем ошибки прогнозы
result = result.assign(forecast = forecast)
result = result.assign(error = result.w0 - result.forecast)
result = result.assign(abs_error = abs(result.error))
result.head()

In [ ]:
# записываем результат в файл
file_result = folder_name + 'Forecast.xlsx'
result.to_excel(excel_writer=file_result, sheet_name='Sheet1',startrow=0, startcol=0)

# Готово! Можно проверять прогнозы.

### Пример R2_score

In [ ]:
# Объяснение на примере, что показывает меткира R2-score

from sklearn.metrics import r2_score

y_true = [0.1, 0.0, 2.0, 1.0, 0.0, 0.0, 2.0, 2.0]
y_pred = [0.9, 0.1, 1.7, 1.0, 0.1, 0.1, 2.5, 1.5]

r2_score (y_true, y_pred)

# Обучаем сеть с перебором параметров

In [ ]:
# Загружаем данные из дампов
X = joblib.load('x.pkl')
y = joblib.load('y.pkl')
max_train = joblib.load('max_train.pkl')
vectorizer = joblib.load('vectorizer.pkl')

In [ ]:
## Fix random seed for reproducibility
seed = 24
np.random.seed(seed)

# разбиваем данные на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

## General and CV options
n_iter = 10
n_jobs = -1
cv = 5
max_iter = 200
batch_size = 'auto'

## Parameters for MLP
neurons_output = max(1, y_train.shape[1] if len(y_train.shape) > 1 else 1)
sumInOutSize = X_train.shape[1] + neurons_output
maxNeuron = sumInOutSize
minNeuron = int(maxNeuron/2)
input_dim = X_train.shape[1] if len(X_train.shape) > 1 else 1
output_dim = y_train.shape[1] if len(y_train.shape) > 1 else 1
n_layers = 1

## MLPRegressor
MLPR_mdl = MLPRegressor(random_state=seed, max_iter=max_iter, batch_size=batch_size)

# задаем параметры сеты, которые будем перебирать
MLPR_param_distributions = \
    { 
#        "hidden_layer_sizes": [(x,) for x in range(minNeuron,maxNeuron)],
        "hidden_layer_sizes": list(hls),
        "solver": ["lbfgs", "sgd", "adam"],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "activation": ["identity", "logistic", "tanh", "relu"],
        "alpha": list(np.random.uniform(low=0, high=5, size=n_iter)),
        "learning_rate_init": list(10.0 ** np.random.uniform(low=-6, high=-2, size=n_iter)),
        "momentum": list(np.random.uniform(low=1E-2, high=1, size=n_iter)),
        "nesterovs_momentum": [True, False],
        "beta_1": list(np.random.uniform(low=1E-2, high=1, size=n_iter)),
        "beta_2": list(np.random.uniform(low=1E-2, high=1, size=n_iter))
    }
    
#param_grid = ParameterGrid(MLPR_param_distributions)
#grid_size = len(param_grid)
#print(grid_size)

MLPR_random_search = RandomizedSearchCV(estimator=MLPR_mdl, cv=cv, n_iter=n_iter, param_distributions=MLPR_param_distributions, n_jobs=n_jobs)
MLPR_start = time.perf_counter()
MLPR_result = MLPR_random_search.fit(X_train, y_train)
MLPR_end = time.perf_counter()
MLPR_y_pred = MLPR_result.predict(X=X_test)

print('Train MLPR_result.score: ',MLPR_result.score(X_train, y_train))
print('Test MLPR_result.score: ',MLPR_result.score(X_test, y_test))

## Summarize results
print("MLPRegressor")
print("============")
print("Time: {0}".format(MLPR_end - MLPR_start))
print("Score: {0}".format(MLPR_result.best_score_))
print("Parameters: {0}".format(MLPR_result.best_params_))
print("r2_score: {0}".format(sklearn.metrics.r2_score(y_true=y_test, y_pred=MLPR_y_pred)))
print("mean_absolute_error: {0}".format(sklearn.metrics.mean_absolute_error(y_true=y_test, y_pred=MLPR_y_pred)))
print("mean_squared_error: {0}".format(sklearn.metrics.mean_squared_error(y_true=y_test, y_pred=MLPR_y_pred)))
print("median_absolute_error: {0}".format(sklearn.metrics.median_absolute_error(y_true=y_test, y_pred=MLPR_y_pred)))
#print("mean_absolute_percentage_error: {0}".format(mean_absolute_percentage_error(y_true=y_test, y_pred=MLPR_y_pred)))